In [1]:
import sys
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import einsum
from torch.utils.data import DataLoader, Dataset

from sentence_transformers import SentenceTransformer

from ascii_utils import ASCIITransformer, ASCIITokenizer

Using apple metal GPU


In [2]:
# check for GPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using apple metal GPU")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using nvidia GPU")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

Using apple metal GPU


In [3]:
# class ASCIITokenizer:
#     def __init__(self):
#         self.vocab = [chr(i) for i in range(32, 127)] + ['']
#         self.vocab_size = len(self.vocab)
#         self.token_to_char = {i: c for i, c in enumerate(self.vocab)}
#         self.char_to_token = {c: i for i, c in enumerate(self.vocab)}

#     def tokenize(self, text):
#         tokens = [self.char_to_token[c] for c in text if c in self.vocab]
#         return tokens

In [4]:
def pad_ascii_art(ascii_art, max_width=32, max_height=32, pad_char=' '):
    lines = ascii_art.splitlines()
    padded_lines = [line.ljust(max_width, pad_char) for line in lines]
    while len(padded_lines) < max_height:
        padded_lines.append(pad_char * max_width)
    return '\n'.join(padded_lines)

df = pd.read_json("./ascii_data/ascii_art_data.json")
df['width'] = df['text'].apply(lambda x: max(len(line) for line in x.splitlines()))
df['height'] = df['text'].apply(lambda x: len(x.splitlines()))
df['text'] = df['text'].apply(pad_ascii_art)

df = df[ (df['width'] < 32) & (df['height'] < 32) ][['topic', 'text']]
df[:5]

,topic,text
2,aardvarks,_.---._ /\\ \n ./' ...
8,bats,"_ ,_, _ \n / `'=..."
9,bats,"(_ ,_, _) \n / `'-..."
10,bats,=/\ /\= \n / \'._ ...
11,bats,/\ /\ \n / \'._ (...


In [5]:
tokenizer = ASCIITokenizer()

class ASCIIDataLoader(Dataset):
    def __init__(self, df, tokenizer):
        df = df.reset_index()
        self.labels = df['topic']
        tokenized_data = [tokenizer.tokenize(art) for art in df['text']]
        max_length = 32 * 32
        tokenized_data = [tokens + [0] * (max_length - len(tokens)) for tokens in tokenized_data]
        self.asciis = torch.tensor(tokenized_data)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.asciis[idx], self.labels[idx]

dataset = ASCIIDataLoader(df, ASCIITokenizer())
loader = DataLoader(dataset, batch_size=10, shuffle=True)

In [6]:
model = ASCIITransformer().to(device)

: 

In [7]:

sentence_transformer_model="sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(sentence_transformer_model).to(device)
def encode(x):
    return embedding_model.encode(x)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for ascii_text, label in loader:
        # print(label)
        # print(ascii_text)
        ascii_text = ascii_text.to(device)
        embedding = torch.from_numpy(encode(label)).to(device)
        optimizer.zero_grad()
        outputs = model(embedding)
        loss = criterion(outputs.view(-1), ascii_text.view(-1).float())
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [ ]:
out = model(torch.tensor(encode(['aardvark'])).to(device)).cpu()
out = out.detach()  # Detach from the computation graph if not already detached

# Apply softmax to convert logits to probabilities along the appropriate dimension
# Note: softmax is not necessary just to get the indices with argmax, but shown here for completeness if probabilities are needed
probabilities = torch.softmax(out, dim=1)  # Ensure 'dim' is set correctly based on your model's output shape

# Use argmax to get the indices of the max probabilities (class indices)
predicted_indices = torch.argmax(probabilities, dim=1)
print(predicted_indices)
# If 'out' was already flat or needs to be reshaped, adjust accordingly:
predicted_indices = predicted_indices.reshape(32, 32)  # Reshape if your indices are not in this shape yet

# Convert to numpy array if you're working with non-torch functions later
predicted_indices = predicted_indices.numpy()

tensor([73])


RuntimeError: shape '[32, 32]' is invalid for input of size 1